In [3]:
from music21 import *
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

### Features
- chord_names (nominal)
        - Label encoding (C major -> 0, A major -> 13)
        - One-hot encoding?
        - Feature hashing
- roman_numerals

In [ ]:
myChord = chord.Chord(["E-5","C4","G4"])
myChord.show()

In [ ]:
myChord

In [ ]:
pitches = myChord.pitches
pitches

In [ ]:
melody = pitches[-1]
melody

In [ ]:
myChord.sortAscending(inPlace = True)

In [ ]:
bachCorpus = corpus.search('bach', 'composer')
bachCorpus[-23].parse().show()

In [ ]:
#Load corpus of Bach chorales
bachCorpus = corpus.search('bach', 'composer')


for chorale in bachCorpus:
    chorale = chorale.parse()
    print('Processing chorale: {}'.format(chorale.metadata.title))
    sopranoPart = chorale.getElementsByClass('Part')[0]
    
    for keychange in sopranoPart.recurse().getElementsByClass(key.Key):
        #Get key information and offset location
        print(keychange, keychange.offset)


In [ ]:
#Iterate through soprano part and print all key changes
print('Key Changes:')
for keychange in soprano.recurse().getElementsByClass(key.Key):
    #Get key information and offset location
    print(keychange, keychange.offset)

In [ ]:
bachFeatures = pd.DataFrame(np.zeros((len(bach),2)), columns = ['roman_numerals', 'mel_notes'], dtype='object')
bachFeatures

In [ ]:
bachFeatures.dtypes

In [ ]:
k = key.Key('f#')  # 3-sharps minor
rn = roman.RomanNumeral('V', k)
rn.key

In [ ]:
rn.pitchedCommonName

### Feature Extraction

In [2]:
# extract the notes from each piece in the corpus and place in in the raw_note_dataset variable

bachCorpus = corpus.getComposer('bach')
raw_note_dataset = [] # 'raw' because this is the data before we transform it for use in the network
for piece in bachCorpus:
    sample = []
    pieceScore = corpus.parse(piece)  # stream.Score object
    melodyPart = pieceScore.getElementsByClass('Part')[0]  # melody parts are always the first part in the score
    melodyNotes = melodyPart.flat.getElementsByClass('Note')
    
    # DON'T CHANGE n to Note or note (namespace conflict)
    for n in melodyNotes.recurse():  # iterates through all notes in the piece
        pitchClass = n.pitch.pitchClassString
        pitchOctave = n.pitch.octave
        noteDuration = n.duration.quarterLength
        sample.append([pitchClass, pitchOctave, noteDuration]) # appends the next time step to the sample
        
    raw_note_dataset.append(sample)
    
# use this to see exactly what the data structure looks like    
# for sample in dataset:
#     print('SAMPLE')
#     for note in sample:
#         print('\tNOTE')
#         print(f"\t\tPitch class – {note[0]}\n\t\tOctave – {note[1]}\n\t\tDuration – {note[2]}")

In [33]:
pd.DataFrame(dataset)

,0,1,2,3,4,5,6,7,8,9,...,548,549,550,551,552,553,554,555,556,557
0,"[5, 4, 1.0]","[7, 4, 0.5]","[0, 4, 0.5]","[5, 4, 0.5]","[5, 3, 0.5]","[9, 3, 0.5]","[5, 3, 0.5]","[9, 3, 0.5]","[0, 4, 0.5]","[5, 4, 0.25]",...,None,None,None,None,None,None,None,None,None,None
1,"[2, 5, 2.0]","[5, 5, 2.0]","[2, 5, 1.0]","[2, 5, 1.0]","[2, 5, 1.0]","[2, 5, 1.0]","[3, 5, 2.0]","[2, 5, 2.0]","[0, 5, 2.0]","[0, 5, 2.0]",...,None,None,None,None,None,None,None,None,None,None
2,"[9, 4, 1.0]","[9, 4, 1.0]","[5, 4, 1.0]","[7, 4, 1.0]","[9, 4, 1.0]","[5, 4, 1.0]","[4, 4, 1.0]","[2, 4, 1.0]","[9, 4, 1.0]","[9, 4, 1.0]",...,None,None,None,None,None,None,None,None,None,None
3,"[7, 4, 1.0]","[7, 4, 1.0]","[3, 4, 1.0]","[5, 4, 1.0]","[7, 4, 0.5]","[5, 4, 0.5]","[3, 4, 1.0]","[2, 4, 1.0]","[0, 4, 1.0]","[7, 4, 1.0]",...,None,None,None,None,None,None,None,None,None,None
4,"[6, 4, 0.5]","[7, 4, 0.5]","[9, 4, 1.0]","[B, 4, 1.0]","[9, 4, 1.0]","[2, 5, 1.0]","[2, 5, 1.0]","[1, 5, 1.0]","[2, 5, 1.0]","[2, 5, 1.0]",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
429,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
430,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
431,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [20]:
print(len(dataset[:]), len(dataset[:][1]), len(dataset[:][1][0]))

433 46 3


In [ ]:
# one-hot encode the the pitch class and octave features

encoder = OneHotEncoder(raw_note_dataset[:,:,0:2])